In [1]:
'''Vector database:'''

'Vector database:'

In [1]:
# https://prnt.sc/J9vD-gL2WwfC
# https://prnt.sc/ngBASrWXpwxh

In [2]:
!pip install pinecone-client

   ---------------------------------------- 0.0/215.5 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/215.5 kB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 81.9/215.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  215.0/215.5 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 215.5/215.5 kB 1.6 MB/s eta 0:00:00


In [2]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
pc = Pinecone(api_key=os.environ['API_KEY_PINECONE'])

In [6]:
index_name = "pinecone-index"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [7]:
index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1",
)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2",
)

{'upserted_count': 3}

In [8]:
print(index.describe_index_stats())

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
 'total_vector_count': 6}


In [9]:
query_results1 = index.query(
    namespace="ns1", vector=[1.0, 1.5], top_k=3, include_values=True
)

print(query_results1)

{'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, 1.5]},
             {'id': 'vec2', 'score': 0.868243158, 'values': [2.0, 1.0]},
             {'id': 'vec3', 'score': 0.850068152, 'values': [0.1, 3.0]}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}


In [11]:
pc.delete_index(name=index_name)